<a href="https://colab.research.google.com/github/GustavoHSilva07/mineracao_analise_dados/blob/main/TAREFA_BIG_A3_Classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Tarefa de Aprofundamento 3** 
## CASO: Controle de Qualidade de Peças de uma Indústria
---
<br>
<br>





A partir deste notebook você vai criar e avaliar diferentes modelos de classificação para prever a qualidade de peças de uma indústria. 

Em seguida, com base nos seus resultados, responda o **questionário do Moodle**. (**dica**: o questionário é de alternativas e as respontas irão ajudá-lo a verificar os seus resultados ao longo do programa). 

# Basic imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Exploração dos Dados



`pieces` é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

**Objetivo:** Seu objetivo é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. 



In [2]:
df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces1.csv')
df.head()

,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ


## Inspecione os dados

In [3]:
# Verificando o numero de linhas e colunas do conjunto dedados
df.shape

(500, 7)

In [4]:
# Verificando o tipo dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       500 non-null    int64  
 1   A        500 non-null    float64
 2   B        491 non-null    float64
 3   C        494 non-null    float64
 4   D        496 non-null    float64
 5   Quality  500 non-null    object 
 6   Unit     500 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 27.5+ KB


In [5]:
# Verificando valores nulos no conjunto de dados
df.isnull().sum()

id         0
A          0
B          9
C          6
D          4
Quality    0
Unit       0
dtype: int64

## Preparação dos dados

Aqui você vai preparar o dado para o uso nos modelos. Você fará a seleção de atributos, tratamento de valores nulos, o encode dos dados, normalização e a separação dos dados de treinamento e teste.

### Seleção de Atributos

Elimine atributos que não fazem parte do treinamento.

In [6]:
df = df.drop(columns= ['id'] )
df.head()

,A,B,C,D,Quality,Unit
0,4.9,3.1,1.5,0.1,Reject,SP
1,4.8,3.4,1.6,0.2,Reject,SP
2,6.7,2.5,5.8,1.8,Refurbish,RJ
3,7.6,3.0,6.6,2.1,Refurbish,RJ
4,4.9,3.1,1.5,0.1,Reject,RJ


### Tratamento de Dados Ausentes

Verifique as presença de valores ausentes e, se houverem, subtitua pelo valor médio dos atributos.

In [7]:

df.B = df.B.fillna(df.B.mean())
df.C = df.C.fillna(df.C.mean())
df.D = df.D.fillna(df.D.mean())

df.isnull().sum()

A          0
B          0
C          0
D          0
Quality    0
Unit       0
dtype: int64

### Hot encode

Faça os *Hot Encodes* **necessários** dos dados. Lembre-se de verificar se de fato todos os dados categóricos precisam dessa transformação.

In [8]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore') 
hot_encode = hot_encode.fit(df[['Unit']])

display(hot_encode.categories_)

df = pd.concat([df, pd.DataFrame(hot_encode.transform(df[['Unit']]).toarray(), columns = hot_encode.categories_)], axis=1)
df = df.drop(columns= ['Unit'] )
df = df.rename(columns={('BH',): 'BH', ('RJ',): 'RJ', ('SP',): 'SP'})
df.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,Quality,BH,RJ,SP
0,4.9,3.1,1.5,0.1,Reject,0.0,0.0,1.0
1,4.8,3.4,1.6,0.2,Reject,0.0,0.0,1.0
2,6.7,2.5,5.8,1.8,Refurbish,0.0,1.0,0.0
3,7.6,3.0,6.6,2.1,Refurbish,0.0,1.0,0.0
4,4.9,3.1,1.5,0.1,Reject,0.0,1.0,0.0


### Normalize os dados

Normalize os dados com o `StandardScaler` (melhor) ou a função `scale`.

$$ z_i = \frac{x_i - \bar{x}}{\sigma(x)}$$

E não esqueça de excluir dados não úteis ao treinamento. Depois de normalizar os dados os valores devem apresentar média próxima de 0 e desvio padrão próximo de 1.

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df.drop(columns= ['Quality'] ))
X = scaler.transform(df.drop(columns= ['Quality']))

X[:5]

array([[-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
        -0.57735027,  1.1100193 ],
       [-1.26113242,  0.7514042 , -1.25881814, -1.34683562, -0.65777257,
        -0.57735027,  1.1100193 ],
       [ 1.06613215, -1.34088009,  1.1581686 ,  0.73466146, -0.65777257,
         1.73205081, -0.90088523],
       [ 2.16852062, -0.17849993,  1.61854703,  1.12494216, -0.65777257,
         1.73205081, -0.90088523],
       [-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
         1.73205081, -0.90088523]])

(Moodle) **Q1**. Após todas as transformações a soma dos valores absolutos ( `np.abs()` ) dos atributos referentes ao hot encode dos *BH*, *RJ* e *SP* são respectivamente?

*Empregue esta pergunta como ponto de checagem das suas transformações. Se não encontrar os seus valores dentre as alternativas há provavelmente um erro e você deve revisar as transformações.*

In [10]:
BH = np.abs(X[:,4]).sum() 
RJ = np.abs(X[:,5]).sum() 
SP = np.abs(X[:,6]).sum() 

print(BH)
print(RJ)
print(SP)

459.12525524087647
433.0127018922194
497.2886485734416


### Conjuntos de Treinamento e Teste

Separe os dados de treinamento e teste (30%, estratificados pela variável objetivo e com random_state = 1984). Não empregue outros parâmetros não solicitados.


In [11]:
from sklearn.model_selection import train_test_split

X = X
y = df.Quality

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state=1984)

print( X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(350, 7) (150, 7) (350,) (150,)


# K-Vizinhos mais Próximos

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para encontrar o melhor modelo entre os valores de $k$ de $4$ a $11$ e empregando as métricas euclidiana e manhattan. Verifique o modelo obtidos e o `classification_report` desse modelo.

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [12]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = KNeighborsClassifier()
param_grid = {'n_neighbors':[4,5,6,7,8,9,10,11], 'metric':['euclidean', 'manhattan']}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()


KNeighborsClassifier(metric='manhattan', n_neighbors=7)

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.88      0.98      0.93        47
   Refurbish       0.98      0.89      0.93        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.95       150
   macro avg       0.95      0.96      0.95       150
weighted avg       0.96      0.95      0.95       150




(Moodle) **Q2** Qual o melhor modelo de Knn obtido e qual a sua acuracidade? 

(Moodle) **Q3** Qual classe teve os elementos mais erroneamente classificados? 

# Regressão Logística

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar o modelo de regressão logística. Empregue somente os parâmetros padrão do `sciki-learn` para regressão logística, para isso basta empregar `param_grid = {}`. Verifique os resultados obtidos desse modelo com o `classification_report`. 

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [ ]:
from sklearn.linear_model import LogisticRegression
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [14]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = LogisticRegression()
param_grid = {}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()


LogisticRegression()

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.92      1.00      0.96        47
   Refurbish       1.00      0.93      0.96        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.97       150
   macro avg       0.97      0.98      0.97       150
weighted avg       0.98      0.97      0.97       150




(Moodle) **Q4** Qual a acuracidade do modelo de regressão logística e qual classe teve mais falsos positivos? 

# Aplicando o melhor Modelo

Empregue o melhor modelo obtido acima para estimar a qualidade das peças das 10 peças do conjunto abaixo.

In [16]:
df_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces_new.csv')
df_new 

,id,A,B,C,D,Unit
0,182,5.5,2.6,4.4,1.2,SP
1,345,5.8,2.7,4.1,1.0,SP
2,42,5.1,3.5,1.4,0.2,SP
3,37,6.3,2.5,4.9,1.5,BH
4,61,6.0,3.0,4.8,1.8,RJ


## Preparação dos Dados

Lembre-se, as mesmas operações feitas com os dados de treinamento precisarão ser feitas aqui também. 

### Seleção de Atributos 

In [17]:
df_new = df_new.drop(columns= ['id']  )

### Hot encode  

In [18]:
hot_encode = hot_encode.fit(df_new[['Unit']])

display(hot_encode.categories_)

df_new = pd.concat( [ df_new.drop(columns='Unit'), pd.DataFrame(hot_encode.transform(df_new[['Unit']]).toarray(), columns = list( hot_encode.categories_[0] )) ], axis=1 ) 
df_new.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,BH,RJ,SP
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0


### Normalize os dados

In [20]:
X_new = scaler.transform(df_new)

X_new 

array([[-0.40371916, -1.10840406,  0.35250636, -0.04589995, -0.65777257,
        -0.57735027,  1.1100193 ],
       [-0.03625633, -0.87592802,  0.17986445, -0.30608708, -0.65777257,
        -0.57735027,  1.1100193 ],
       [-0.89366959,  0.98388023, -1.37391274, -1.34683562, -0.65777257,
        -0.57735027,  1.1100193 ],
       [ 0.57618171, -1.34088009,  0.64024287,  0.34438075,  1.5202823 ,
        -0.57735027, -0.90088523],
       [ 0.20871889, -0.17849993,  0.58269557,  0.73466146, -0.65777257,
         1.73205081, -0.90088523]])

## Predição

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_new)

df_new['Prediction'] = y_pred

df_new

,A,B,C,D,BH,RJ,SP,Prediction
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0,Accept
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0,Accept
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0,Reject
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0,Accept
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0,Refurbish


(Moodle) **Q5** Quais as classes obtidas com o melhor modelo para os cinco novos casos? 

# Apêndice: Árvore de Decisão (RESOLVIDO)

Este é apenas um exemplo empregando o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar um modelo de Árvore de Decisão com diferentes parâmetros. Você irá estudar mais detalhtes do modelo de Árvore de Decisão na próxima trilha. Mas ele é um classificador, assim como o modelo de K-Vizinhos mais Próximos e a Regressão Logística, e o uso desse estimador segue os mesmos moldes desses estimadores. Você pode assim aplicá-lo do mesmo modo. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [ None, 3 ]}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

DecisionTreeClassifier()

Detailed classification report:

              precision    recall  f1-score   support

      Accept       1.00      1.00      1.00        47
   Refurbish       1.00      1.00      1.00        55
      Reject       1.00      1.00      1.00        48

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150


